In [26]:
import numpy as np

In [27]:
import ufl

In [28]:
from dolfinx import fem, io, mesh, plot

In [29]:
from ufl import ds, dx, grad, inner

In [30]:
from mpi4py import MPI

In [31]:
from petsc4py.PETSc import ScalarType

In [32]:
msh = mesh.create_rectangle(comm=MPI.COMM_WORLD, points=((0.0, 0.0), (2.0, 1.0)), n=(32, 16), cell_type=mesh.CellType.triangle)

In [33]:
V = fem.FunctionSpace(msh, ("Lagrange", 1))

In [34]:
facets = mesh.locate_entities_boundary(msh, dim=1, marker=lambda x: np.logical_or(np.isclose(x[0], 0.0), np.isclose(x[0], 2.0)))

In [35]:
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

In [36]:
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [37]:
u = ufl.TrialFunction(V)

In [38]:
v = ufl.TestFunction(V)

In [39]:
x = ufl.SpatialCoordinate(msh)

In [40]:
f = 10*ufl.exp(-((x[0] - 0.5)**2 + (x[1] - 0.5)**2) / 0.002)

In [41]:
g = ufl.sin(5 * x[0])

In [42]:
a = inner(grad(u), grad(v)) * dx

In [43]:
L = inner(f, v) * dx + inner(g, v) * ds

In [44]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}) # precondicionadores lu, gmrs (depende de la forma de la matriz)

In [45]:
uh = problem.solve()

In [46]:
with io.XDMFFile(msh.comm, "poisson.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)

In [47]:
!(ls)

LICENSE  README.md  Untitled.ipynb  poisson.h5	poisson.xdmf


In [48]:
try:
    import pyvista
    cells, types, x = plot.create_vtk_mesh(V)
    grid = pyvista.UnstructuredGrid(cells, types, x)
    grid.point_data["u"] = uh.x.array.real
    grid.set_active_scalars("u")
except ModuleNotFoundError:
    print("pyvista is required")

pyvista is required
